# Finland

In [12]:
import pandas as pd
import geopandas as gpd

pd.options.mode.chained_assignment = None

## Fetch results

In [9]:
df = pd.read_csv('../../data/raw/fin.csv', encoding='latin1', skiprows=1, na_values='-')

In [10]:
df = df[df.Område.str.split(' ', expand=True)[0].apply(len) == 3]
df = df[df['Valdeltagande Kön totalt'].notnull()]
df.columns = ['municipality', 'year', 'turnout']
df = df.drop('year', axis=1)
df['id'] = df.municipality.str[:3]
df.municipality = df.municipality.str[4:]
df.municipality = df.municipality.str.replace('S:t Karins', 'St Karins')

## Merge

In [30]:
geo = gpd.read_file('../../data/shapefiles/fin/SuomenKuntajako_2019_10k.shp')

In [32]:
geo = geo.to_crs(epsg=4326)

In [35]:
geo = geo.merge(df, left_on='NATCODE', right_on='id', how='left')

## Export

In [39]:
geo = geo[['NAMESWE', 'turnout', 'geometry']]
geo = geo.rename(columns={'NAMESWE': 'name'})

In [41]:
geo['country'] = 'fin'

In [42]:
geo.to_file('../../data/processed/fin')